
## Build user-based recommendation model for Amazon

In [1]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('Amazon - Movies and TV Ratings.csv')

In [3]:
df1.head()

,user_id,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6,Movie7,Movie8,Movie9,...,Movie197,Movie198,Movie199,Movie200,Movie201,Movie202,Movie203,Movie204,Movie205,Movie206
0,A3R5OBKS7OM2IR,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AH3QC2PC1VTGP,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3LKP6WPMP9UKX,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AVIY68KEPQ5ZD,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1CV1WROP5KTTW,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Exploratory Data Analysis
## Which movies have maximum views/ratings
* Printing First 10 movies having maximum views/ratings

In [4]:
movie_list = df1.notna().sum(axis=0).sort_values(ascending = False).head(10)
movie_list.index.tolist()[1:]

['Movie127',
 'Movie140',
 'Movie16',
 'Movie103',
 'Movie29',
 'Movie91',
 'Movie92',
 'Movie89',
 'Movie158']

## What is the average rating for each movie ? 
* Printing Average rating of first 10 movies

In [5]:
df2 = df1.drop('user_id',axis=1).mean()
df3 = df1.notna().sum(axis=0)
df3.drop(df3.index[0],inplace=True)

In [6]:
df_WithAvgRatingAndVotingCount = pd.concat([df2, df3], axis=1, sort=False)
df_WithAvgRatingAndVotingCount.columns=['AvgRating','RatingCount']
df_WithAvgRatingAndVotingCount.head(10)

,AvgRating,RatingCount
Movie1,5.000000,1
Movie2,5.000000,1
Movie3,2.000000,1
Movie4,5.000000,2
Movie5,4.103448,29
Movie6,4.000000,1
Movie7,5.000000,1
Movie8,5.000000,1
Movie9,5.000000,1
Movie10,5.000000,1


In [7]:
C = df_WithAvgRatingAndVotingCount['AvgRating'].mean() #Mean vote average on the whole dataset
m = df_WithAvgRatingAndVotingCount['RatingCount'].quantile(0.85) # Minimum number of votes required to be listed

print(f'Average rating on dataset {C} and Minimum no. of votes Required {m}')

Average rating on dataset 4.448436665448387 and Minimum no. of votes Required 9.0


In [8]:
def wtdrt(x,m=m,C=C):
    V = x['RatingCount']
    R = x['AvgRating']
    
    return (R*(V/(V+m)) + (C*(m/(V+m))))

In [9]:
df_MovieListConsidredForScore = df_WithAvgRatingAndVotingCount.loc[df_WithAvgRatingAndVotingCount['RatingCount']>=m]

In [10]:
df_MovieListConsidredForScore['Score'] = df_MovieListConsidredForScore.apply(wtdrt,axis=1)

## Top5 movies with maximum Score

In [11]:
df_MovieListConsidredForScore['Score'].sort_values(ascending=False).head(5)

Movie140    4.828000
Movie29     4.793793
Movie158    4.773812
Movie92     4.745781
Movie206    4.728906
Name: Score, dtype: float64

## Top5 Movies with least Audience

In [12]:
movie_list_with_least_audience = \
df_WithAvgRatingAndVotingCount[df_WithAvgRatingAndVotingCount['RatingCount'] == 1]['AvgRating'].head().sort_values \
(ascending=True)
movie_list_with_least_audience.index.tolist()

['Movie3', 'Movie6', 'Movie1', 'Movie2', 'Movie7']

# Build Recommendation Model
## Divide the data into training and test data